In [1]:
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

InteractiveShell.ast_node_interactivity = "all"

In [2]:
## SPLIT oversized csv
# ! cd data
# ! split -l 4000000 311.csv
# ! mv xaa 311a.csv
# ! mv xab 311b.csv

In [3]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    df311a = pd.read_csv('data/311a.csv', skiprows=None, nrows=None, index_col=0)
    df311b = pd.read_csv('data/311b.csv', skiprows=None, nrows=None, index_col=0, header=None)

df311b.columns = df311a.columns
df311 = pd.concat([df311a, df311b])

CPU times: user 35.7 s, sys: 2.1 s, total: 37.8 s
Wall time: 37 s


In [4]:
df311 = df311.rename(columns={
    'Unique Key': 'Key',
    'Created Date': 'Created',
    'Closed Date': 'Closed',
    'Complaint Type': 'Complaint',
    'Location Type': 'Building_type',
    'Incident Zip': 'Zip', # Must leave as float because some na
    'Incident Address': 'Address',
    'Street Name': 'Street',
    'Address Type': 'Address_type',
    'Resolution Description': 'Description',
    'Borough': 'Boro'
})

In [5]:
%%time
df311.Created = pd.to_datetime(df311.Created, format='%m/%d/%Y %I:%M:%S %p')
df311.Closed = pd.to_datetime(df311.Closed, format='%m/%d/%Y %I:%M:%S %p')

CPU times: user 33.8 s, sys: 541 ms, total: 34.3 s
Wall time: 33.9 s


In [6]:
pd.options.display.max_columns = 10
pd.options.display.max_rows = 2
df311

,Key,Created,Closed,Complaint,Building_type,...,Status,Description,Boro,Latitude,Longitude
0,45531130,2020-02-02 06:09:17,NaT,HEAT/HOT WATER,RESIDENTIAL BUILDING,...,Open,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
...,...,...,...,...,...,...,...,...,...,...,...
6019842,44063737,2019-10-15 11:29:01,2019-10-15 21:19:45,PAINT/PLASTER,RESIDENTIAL BUILDING,...,Closed,The Department of Housing Preservation and Dev...,MANHATTAN,40.724749,-73.975269


In [7]:
pd.options.display.max_rows = None

pd.options.display.float_format = '{:.4f}'.format
df311[['Latitude','Longitude']].describe()

df311.Address_type.value_counts()
df311 = df311.drop(columns=['Address_type'])

df311.Status.value_counts()

Building_type = {
    'RESIDENTIAL BUILDING': 'Residence',
    'Residential Building': 'Residence',
    'Apartment':            'Apartment',
    'Building-Wide':        'Buildingwide',
    'Public Area':          'Public',
}
df311.Building_type = df311.Building_type.map(Building_type)
df311.Building_type.value_counts()

Complaint = {
    'HEAT/HOT WATER':         'Hotwater',
    'HEATING':                'Heat',
    'PLUMBING':               'Plumbing',
    'GENERAL CONSTRUCTION':   'Construction',
    'UNSANITARY CONDITION':   'Unsanitary',
    'PAINT - PLASTER':        'Paint',
    'PAINT/PLASTER':          'Paint',
    'ELECTRIC':               'Electric',
    'NONCONST':               'Nonconstruction',
    'DOOR/WINDOW':            'Door',
    'WATER LEAK':             'Leak',
    'GENERAL':                'General',
    'FLOORING/STAIRS':        'Stairs',
    'APPLIANCE':              'Appliance',
    'HPD Literature Request': 'Literature',
    'SAFETY':                 'Safety',
    'OUTSIDE BUILDING':       'Outside',
    'ELEVATOR':               'Elevator',
    'Unsanitary Condition':   'Unsanitary',
    'CONSTRUCTION':           'Construction',
    'General':                'General',
    'Safety':                 'Safety',
    'STRUCTURAL':             'Structural',
    'Plumbing':               'Plumbing',
    'AGENCY':                 'Agency',
    'VACANT APARTMENT':       'Vacancy',
    'Outside Building':       'Outside',
    'Appliance':              'Appliance',
    'Mold':                   'Mold',
    'Electric':               'Electric',
}
df311.Complaint = df311.Complaint.map(Complaint)
pd.options.display.max_rows = 10
df311.Complaint.value_counts()

,Latitude,Longitude
count,5939172.0000,5939172.0000
mean,40.7542,-73.9199
std,0.0909,0.0568
min,40.4993,-74.2531
25%,40.6712,-73.9520
50%,40.7570,-73.9225
75%,40.8396,-73.8902
max,40.9129,-73.7008


ADDRESS    5935078
Name: Address_type, dtype: int64

Closed         5886253
Open            133220
In Progress        364
Assigned             4
Pending              2
Name: Status, dtype: int64

Residence       5967008
Apartment             5
Buildingwide          4
Public                2
Name: Building_type, dtype: int64

Hotwater        1261574
Heat             887850
Plumbing         711141
Paint            707695
Construction     505941
                 ...   
Elevator           6725
Structural           16
Agency                9
Vacancy               6
Mold                  1
Name: Complaint, Length: 21, dtype: int64

In [8]:
%%time

# df311.City.isna().sum()
# df311.index[df311.City.isna()]
# df311.loc[2300:2350,'City'].str.title() # Yes, .str.title handles NaN

df311.City = df311.City.str.title()
pd.options.display.max_rows = 10
df311.City.value_counts()

CPU times: user 4.07 s, sys: 157 ms, total: 4.23 s
Wall time: 3.87 s


Brooklyn         2026580
Bronx            1860522
New York         1204417
Staten Island     101553
Jamaica            97792
                  ...   
Glen Oaks            764
Queens               491
Floral Park          287
Breezy Point         269
New Hyde Park        112
Name: City, Length: 47, dtype: int64

In [9]:
%%time
df311.loc[df311.Boro=='Unspecified', 'Boro'] = np.nan

pd.options.display.max_rows = None
df311.Boro.value_counts()

CPU times: user 1.69 s, sys: 16.4 ms, total: 1.7 s
Wall time: 1.54 s


BROOKLYN         1739886
BRONX            1617956
MANHATTAN        1055225
QUEENS            645971
STATEN ISLAND      87584
Name: Boro, dtype: int64

In [10]:
%time df311.to_pickle('data/311.pkl')

CPU times: user 6.51 s, sys: 2.35 s, total: 8.86 s
Wall time: 10 s


In [11]:
%%time

df311.isna().sum()

pd.options.display.float_format = None
pd.options.display.max_columns = 10
pd.options.display.max_rows = 2
df311

pd.options.display.float_format = '{:.1f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df311.describe().T
df311.describe(include='object').T

CPU times: user 17.2 s, sys: 709 ms, total: 17.9 s
Wall time: 14.2 s


,count,unique,top,freq
Complaint,6019843,21,Hotwater,1261574
Building_type,5967019,4,Residence,5967008
Address,5967018,182600,34 ARDEN STREET,14298
Street,5967018,6825,GRAND CONCOURSE,92450
City,5939569,47,Brooklyn,2026580
Status,6019843,5,Closed,5886253
Description,6012017,340,The Department of Housing Preservation and Dev...,1698990
Boro,5146622,5,BROOKLYN,1739886


### Next Steps
1. Investigate whether `Address_type == ADDRESS` versus `== NaN` is meaningful.
   If so, do not drop the `Address_type` column